typeEchantillon : 
- fixe pour utiliser tailleEchantillon
- variable pour utiliser nombre,increment,nombreInitial
- total pour utiliser tout le lexique

In [1]:
typeEchantillon="fixe"
tailleEchantillon=30000
nombre=1000
increment=250
nombreInitial=20

##Importations
- codecs pour les encodages
- pandas et numpy pour les calculs sur tableaux
- matplotlib pour les graphiques
- itertools pour les itérateurs sophistiqués (paires sur liste, ...)

In [2]:
# -*- coding: utf8 -*-
import codecs
import features
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools as it
import pickle
#%pylab inline
#pd.options.display.mpl_style = 'default'
debug=False

###Préparation des matrices de traits

In [3]:
features.add_config('bdlexique.ini')
fs=features.FeatureSystem('phonemes')
#fs.supremum.concept.extent

In [4]:
bdlexiqueIn = unicode(u"èò")
bdlexiqueNum = [ord(char) for char in bdlexiqueIn]
neutreOut = unicode(u"EO")
neutralise = dict(zip(bdlexiqueNum, neutreOut))

In [5]:
def recoder(chaine,table=neutralise):
    if type(chaine)==str:
        temp=unicode(chaine.decode('utf8')).translate(table)
        result=temp.encode('utf8')
    elif type(chaine)==unicode:
        result=chaine.translate(table)
    return result

In [6]:
filePrefix="MGC-"

#Préparation du tableau de VERBES

###Lecture du lexique

In [7]:
nomLexique="/Users/gilles/Copy/Python/phonemisation/bdlexique-PDM-Resync.txt"
bdlexique=pd.read_csv(nomLexique,sep=";",names=["ortho","phono","ext","cs","ms","vs","lexeme","L23","lemmeFrantext","lemmeFilms","formeFrantext","formeFilms"],encoding="utf8")

###Définition des cases
- principales pour le paradigme courant
- secondaires pour les cases rares
- totales pour le tout

In [8]:
casesPrincipales= [
        'inf', 'pi1S', 'pi2S', 'pi3S', 'pi1P', 'pi2P', 'pi3P', 'ii1S',
        'ii2S', 'ii3S', 'ii1P', 'ii2P', 'ii3P', 
        'fi1S', 'fi2S', 'fi3S', 'fi1P', 'fi2P',
        'fi3P', 'pI2S', 'pI1P', 'pI2P', 'ps1S', 'ps2S', 'ps3S', 'ps1P',
        'ps2P', 'ps3P', 
        'pc1S', 'pc2S', 'pc3S', 'pc1P', 'pc2P', 'pc3P', 'pP',
        'ppMS', 'ppMP', 'ppFS', 'ppFP'
            ]
casesSecondaires= [
       'ai1S', 'ai2S', 'ai3S', 'ai1P', 'ai2P', 'ai3P', 'is1S', 'is2S', 'is3S', 'is1P', 'is2P', 'is3P'
            ]
casesTotales=casesPrincipales+casesSecondaires
listeCases=casesTotales

###Création de la colonne de fréquence *freq*
- fréquence positive : 10.f
- fréquence nulle : 5
- fréquence inexistante : 1

In [9]:
bdlexique["freq"]=bdlexique[bdlexique["formeFrantext"]!="***"]["formeFrantext"].astype(float)*10
bdlexique.loc[bdlexique["formeFrantext"]=="0","freq"]=5
bdlexique.loc[bdlexique["formeFrantext"]=="***","freq"]=1

###Elimination des colonnes inutiles
- indicateur L23
- fréquences brutes

In [10]:
inutilesColonnes=["L23","lemmeFrantext","lemmeFilms","formeFrantext","formeFilms"]
for colonne in inutilesColonnes:
    del bdlexique[colonne]

###Extraction des formes verbales dans une structure *verbes*

In [11]:
verbes=bdlexique[bdlexique["cs"].isin(["V","K"])].copy()

In [12]:
verbes["phono"]=verbes["phono"].apply(lambda x: recoder(x))

In [13]:
#verbes["phono"]
print recoder(u"tOtòtètE")

tOtOtEtE


###Ajout d'une colonne *prob* pour la fréquence relative

In [14]:
verbes["prob"]=verbes["freq"]/verbes["freq"].sum()

###Ajout d'une colonne *case*
- remplir les *ms* vides pour permettre la concaténation sans erreur
- remplir les *vs* vides des participes passés par "pp" pour permettre l'identification de la case
- *case* est la concaténation de *vs* et *ms*

In [15]:
verbes["ms"]=verbes["ms"].fillna("")
verbes["vs"]=verbes["vs"].fillna("pp")
verbes["case"]=verbes["vs"]+verbes["ms"]

#Echantillonage

##Assembler les échantillons correspondant à une étape n

In [16]:
def assemblerExtrait(nombre):
    return verbes.ix[np.sort(np.concatenate(tirages[0:nombre]))]

###Tirage incrémental des formes disponibles
- increment : taille de chaque tirage
- nombre : nombre de tirages

np.random.choice donne une liste d'index de formes tirées dans l'ordre du tirage

on découpe la liste en morceaux de la taille de l'incrément

In [17]:
if typeEchantillon=="variable":
    tirage=np.random.choice(verbes.index,size=nombre*increment,p=verbes["prob"],replace=False)
    tirages=[tirage[increment*x:increment*(x+1)] for x in range(len(tirage)/increment+(len(tirage)%increment!=0))]
    extrait=assemblerExtrait(nombreInitial)
elif typeEchantillon=="fixe":
    tirage=np.random.choice(verbes.index,size=tailleEchantillon,p=verbes["prob"],replace=False)
    extrait=verbes.ix[np.sort(tirage)]
elif typeEchantillon=="total":
    extrait=verbes

In [18]:
#extrait=assemblerExtrait(nEchantillons)

In [19]:
paradigmes=pd.pivot_table(extrait, values='phono', index=['lexeme'], columns=['case'], aggfunc=lambda x: ",".join(x)).reset_index().reindex()

In [20]:
def diff(mot1,mot2):
    result=[]
    diff1=""
    diff2=""
    same=""
    vide="."
    lmax=max(len(mot1),len(mot2))
    lmin=min(len(mot1),len(mot2))
    for index in range(lmax):
        if index < lmin:
            if mot1[index]!=mot2[index]:
                diff1+=mot1[index]
                diff2+=mot2[index]
                same+=vide
            else:
                same+=mot1[index]
                diff1+=vide
                diff2+=vide
        elif index < len(mot1):
            diff1+=mot1[index]
        elif index < len(mot2):
            diff2+=mot2[index]
    diff1=diff1.lstrip(".")
    diff2=diff2.lstrip(".")
#    return (same,diff1,diff2,diff1+"_"+diff2)
    return (diff1+"-"+diff2)

def rowDiff(row, patrons):
    result=diff(row[0],row[1])
    if not result in patrons:
        patrons[result]=(formesPatron(),formesPatron())
    patrons[result][0].ajouterFormes(row[0])
    patrons[result][1].ajouterFormes(row[1])
    return (result[0],result[1])


In [21]:
def patron2regexp(morceaux):
    result="^"
    for morceau in morceaux:
        if morceau=="*":
            result+="(.*)"
        elif len(morceau)>1:
            result+="(["+morceau+"])"
        else:
            result+=morceau
    result+="$"
    result=result.replace(")(","")
    return result

In [22]:
def remplacementSortie(sortie):
    n=1
    nsortie=""
    for lettre in sortie:
        if lettre==".":
            nsortie+="\g<%d>"%n
            n+=1
        else:
            nsortie+=lettre
    return nsortie

In [23]:
class formesPatron:
    '''
    Accumulateur de formes correspondant à un patron pour calcul de la Généralisation Minimale (cf. MGL)
    '''
    def __init__(self):
        self.formes=[]

#    def __repr__(self):
#        return ','.join(self.calculerGM())
        
    def ajouterForme(self,forme):
        self.formes.append(forme)
        
    def calculerGM(self):
        minLongueur=len(min(self.formes, key=len))
        maxLongueur=len(max(self.formes, key=len))
        if debug: print minLongueur, maxLongueur
        positions=[]
        if maxLongueur>minLongueur:
            positions.append("*")
        for i in xrange(minLongueur, 0, -1):
            phonemes=set([x[-i] for x in self.formes])
            if debug: print phonemes
            if "." in phonemes:
                positions.append(".")
            else:
                positions.append("".join(fs.lattice[phonemes].extent))
        return patron2regexp(positions)

class pairePatrons:
    '''
    Accumulateur de triplets (f1,f2,patron) correspondant à une paire pour calcul des Généralisations Minimales (cf. MGL)
    '''
    def __init__(self,case1,case2):
        self.patrons1={}
        self.patrons2={}
        self.case1=case1
        self.case2=case2

#    def __repr__(self):
#        return ','.join(self.calculerGM())
        
    def ajouterFormes(self,forme1,forme2,patron):
#        print forme1,forme2,patron
        patron12=patron
        (pat1,pat2)=patron.split("-")
        patron21=pat2+"-"+pat1
#        print patron12,patron21
        if not patron12 in self.patrons1:
            self.patrons1[patron12]=formesPatron()
        self.patrons1[patron12].ajouterForme(forme1)
        if not patron21 in self.patrons2:
            self.patrons2[patron21]=formesPatron()
        self.patrons2[patron21].ajouterForme(forme2)
        
        
    def calculerGM(self):
        resultat1={}
        for patron in self.patrons1:
            if debug: print "patron1", patron
            resultat1[patron]=self.patrons1[patron].calculerGM()
        resultat2={}
        for patron in self.patrons2:
            if debug: print "patron2", patron
            resultat2[patron]=self.patrons2[patron].calculerGM()
        return (resultat1,resultat2) 

#Classe pour la gestion des patrons, des classes et des transformations

In [24]:
class paireClasses:
    def __init__(self,case1,case2):
        self.case1=case1
        self.case2=case2
        self.nom=case1+"-"+case2
        self.classes1=classesPaire(case1,case2)
        self.classes2=classesPaire(case2,case1)

    def ajouterPatron(self,n,patron,motif):
        if n==1:
            self.classes1.ajouterPatron(patron,motif)
        elif n==2:
            self.classes2.ajouterPatron(patron,motif)
        else:
            print "le numéro de forme n'est pas dans [1,2]",n

    def ajouterPaire(self,forme1,forme2):
        self.classes1.ajouterPaire(forme1,forme2)
        self.classes2.ajouterPaire(forme2,forme1)
        
    def calculerClasses(self):
        return(self.classes1,self.classes2)

    
class classesPaire:
    '''
    Gestion des patrons, des classes et des transformations
    
    ajouterPatron : ajoute un patron et son motif associé (MGL)
    ajouterPaire : ajoute une paire de formes, calcule la classe de la forme1 et la règle sélectionnée
    sortirForme : cacule les formes de sortie correspondant à la forme1 avec leurs coefficients respectifs
    '''
    def __init__(self,case1,case2):
        self.case1=case1
        self.case2=case2
        self.nom=case1+"-"+case2
        self.classe={}
        self.nbClasse={}
        self.patrons={}
        self.entree={}
        self.sortie={}
    
    def ajouterPatron(self,patron,motif):
        self.patrons[patron]=motif
        (entree,sortie)=patron.split("-")
        self.entree[patron]=entree.replace(u".",u"(.)")
        self.sortie[patron]=remplacementSortie(sortie)
    
    def ajouterPaire(self,forme1,forme2):
        '''
        on calcule la classe de la paire idClasseForme et la règle sélectionnée
        on incrémente le compteur de la classe et celui de la règle sélectionnée à l'intérieur de la classe
        '''
        classeForme=[]
        regleForme=""
        for patron in self.patrons:
            if re.match(self.patrons[patron],forme1):
                classeForme.append(patron)
                '''
                le +"$" permet de forcer l'alignement à droite pour les transformations suffixales
                '''
                if forme2==re.sub(self.entree[patron]+"$",self.sortie[patron],forme1):
                    regleForme=patron
        idClasseForme=", ".join(classeForme)
        if not idClasseForme in self.classe:
            self.classe[idClasseForme]={}
            self.nbClasse[idClasseForme]=0
        if not regleForme in self.classe[idClasseForme]:
            self.classe[idClasseForme][regleForme]=0
        self.nbClasse[idClasseForme]+=1
        self.classe[idClasseForme][regleForme]+=1

    def sortirForme(self,forme):
        classeForme=[]
        sortieForme={}
        for patron in self.patrons:
            if re.match(self.patrons[patron],forme):
                classeForme.append(patron)
        if classeForme:
            idClasseForme=", ".join(classeForme)
            if idClasseForme in self.nbClasse:
                nTotal=self.nbClasse[idClasseForme]
                for patron in self.classe[idClasseForme]:
                    sortie=re.sub(self.entree[patron]+"$",self.sortie[patron],forme)
                    sortieForme[sortie]=float(self.classe[idClasseForme][patron])/nTotal
            else:
                print forme,
                print "pas de classe",idClasseForme,
                print "%.2f par forme de sortie" % (float(1)/len(classeForme))
                nTotal=len(classeForme)
                for patron in classeForme:
                    sortie=re.sub(self.entree[patron]+"$",self.sortie[patron],forme)
                    sortieForme[sortie]=float(1)/nTotal
        else:
            print forme, 
            print "pas de patron"
        return sortieForme
        

##Appliquer la formule de calcul des différences entre chaines à chaque ligne

>si il y a au moins une ligne

>>on applique la différence à la ligne

>>on calcule les deux patrons par suppression des points initiaux

>>on renvoie le groupement par patrons (1&2)

>sinon

>>on renvoie le paradigme vide d'origine

In [25]:
def OLDrapports(paradigme):
    (case1,case2,lexeme)= paradigme.columns.values.tolist()
    patrons=pairePatrons(case1,case2)
    if len(paradigme)>0:
#        for index, row in paradigme.iterrows():
#            patrons.ajouterFormes(row[0],row[1],diff(row[0],row[1]))
        paradigme.apply(lambda x: patrons.ajouterFormes(x[case1],x[case2],diff(x[case1],x[case2])), axis=1)
        (regles1,regles2)=patrons.calculerGM()
    return patrons.calculerGM()

In [26]:
def rapports(paradigme):
    if len(paradigme.columns.values.tolist())==2:
        (case1,lexeme)= paradigme.columns.values.tolist()
        case2=case1
    else:
        (case1,case2,lexeme)= paradigme.columns.values.tolist()
    patrons=pairePatrons(case1,case2)
    classes=paireClasses(case1,case2)
    if len(paradigme)>0:
        paradigme.apply(lambda x: patrons.ajouterFormes(x[case1],x[case2],diff(x[case1],x[case2])), axis=1)
        (regles1,regles2)=patrons.calculerGM()
        for regle in regles1:
            classes.ajouterPatron(1,regle,regles1[regle])
        for regle in regles2:
            classes.ajouterPatron(2,regle,regles2[regle])
        paradigme.apply(lambda x: classes.ajouterPaire(x[case1],x[case2]), axis=1)
    (classes1,classes2)=classes.calculerClasses()
    return (classes1,classes2)

###Dédoubler les lignes avec des surabondances dans *colonne*
>identifier une ligne avec surabondance

>>ajouter les lignes correspondant à chaque valeur

>>ajouter le numéro de la ligne initiale dans les lignes à supprimer

>supprimer les lignes avec surabondance

NB : il faut préparer le tableau pour avoir une indexation qui permette l'ajout des valeurs individuelles et la suppression des lignes de surabondances

In [27]:
def splitCellMates(df,colonne):
    '''
    Calcul d'une dataframe sans surabondance par dédoublement des valeurs
    '''
    test=df.reset_index()
    del test["index"]
    splitIndexes=[]
    for index,ligne in test.iterrows():
        if "," in ligne[colonne]:
            valeurs=set(ligne[colonne].split(","))
            nouvelleLigne=ligne
            for valeur in valeurs:
                nouvelleLigne[colonne]=valeur
                test=test.append(nouvelleLigne,ignore_index=True)
            splitIndexes.append(index)
    if splitIndexes:
        test=test.drop(test.index[splitIndexes])
    return test


##Calculer les rapports entre formes pour chaque paire

>on fait la liste des cases de *paradigmes*

>pour chaque paire du tableau principal

>>si la paire fait partie des cases de *paradigmes*

>>>on calcule le rapport

>>sinon

>>>on signale que qu'une des cases n'est pas représentée

In [28]:
def evaluerEchantillon(paradigmes):
    result={}
    colonnes=paradigmes.columns.values.tolist()
    for paire in it.combinations_with_replacement(listeCases,2):
        if debug: print paire
#        if paire[0]==paire[1]:
#            paireListe=[paire[0]]
#        else:
#            paireListe=list(paire)
        paireListe=list(paire)
        paireListe.append("lexeme")
        if paire[0] in colonnes and paire[1] in colonnes:
            paradigmePaire=paradigmes[paireListe].dropna(thresh=3, axis=0).reindex()
            if paire[0]==paire[1]:
                paireListe[1]="TEMP"
                paradigmePaire.columns=paireListe
            paradigmePaire=splitCellMates(splitCellMates(paradigmePaire,paireListe[0]),paireListe[1])
            result[paire]=rapports(paradigmePaire)
        else:
            result[paire]=("missing pair", paire)
    return result

###Boucle de calcul des analogies pour l'échantillon

In [29]:
%%time
debug=False
resultats=evaluerEchantillon(paradigmes)

CPU times: user 1min 14s, sys: 245 ms, total: 1min 14s
Wall time: 1min 14s


In [30]:
classesFinales={}
for resultat in resultats:
    classesFinales[resultat]=resultats[resultat][0]
    classesFinales[(resultat[1],resultat[0])]=resultats[resultat][1]

In [31]:
with open(filePrefix+typeEchantillon+'-Regles.pkl', 'wb') as output:
   pickle.dump(classesFinales, output, pickle.HIGHEST_PROTOCOL)

In [32]:
classesFinales[('pi2S', 'pi3P')].sortirForme("aswa")

{'aswa': 0.5, u'aswav': 0.5}

In [33]:
with open(filePrefix+typeEchantillon+'-Regles.pkl', 'rb') as input:
    resultatsLecture = pickle.load(input)

In [51]:
resultatsLecture[('fi1P', 'fi2S')].sortirForme(u"kurrô")

{u'kurra': 1.0}

In [44]:
paradigmes[paradigmes["lexeme"]=="courir"]

case,lexeme,ai1P,ai1S,ai2P,ai2S,ai3P,ai3S,fi1P,fi1S,fi2P,...,ppFS,ppMP,ppMS,ppij,ps1P,ps1S,ps2P,ps2S,ps3P,ps3S
806,courir,NaN,kury,NaN,NaN,NaN,kury,kurrô,kurrE,kurre,...,NaN,NaN,kury,NaN,NaN,kur,NaN,kur,kur,NaN


In [36]:
%%time
for forme in listeCases:
    if not pd.isnull(paradigmes[paradigmes["lexeme"]=="manger"][forme].iloc[0]):
        depart=paradigmes[paradigmes["lexeme"]=="manger"][forme].iloc[0]
        print forme,depart
        if depart!="nan":
            for case in listeCases:
                print case,
                if not isinstance(resultatsLecture[(forme, case)],str):
                    print resultatsLecture[(forme, case)].sortirForme(depart),
            print
            print

inf mâZe
inf {u'm\xe2Ze': 1.0} pi1S {u'm\xe2Z': 1.0} pi2S {u'm\xe2Z': 1.0} pi3S {u'm\xe2Z': 1.0} pi1P {u'm\xe2Z\xf4': 1.0} pi2P {u'm\xe2Ze': 1.0} pi3P {u'm\xe2Z': 1.0} ii1S {u'm\xe2ZE': 1.0} ii2S {u'm\xe2ZE': 1.0} ii3S {u'm\xe2ZE': 1.0} ii1P {u'm\xe2Zj\xf4': 1.0} ii2P {u'm\xe2Zje': 1.0} ii3P {u'm\xe2ZE': 1.0} fi1S {u'm\xe2Z6rE': 1.0} fi2S {u'm\xe2Z6ra': 1.0} fi3S {u'm\xe2Z6ra': 1.0} fi1P {u'm\xe2Z6r\xf4': 1.0} fi2P {u'm\xe2Z6re': 1.0} fi3P {u'm\xe2Z6r\xf4': 1.0} pI2S {u'm\xe2Z': 1.0} pI1P {u'm\xe2Z\xf4': 1.0} pI2P {u'm\xe2Ze': 1.0} ps1S {u'm\xe2Z': 1.0} ps2S {u'm\xe2Z': 1.0} ps3S {u'm\xe2Z': 1.0} ps1P {u'm\xe2Zj\xf4': 1.0} ps2P {u'm\xe2Zje': 1.0} ps3P {u'm\xe2Z': 1.0} pc1S {u'm\xe2Z6rE': 1.0} pc2S {u'm\xe2Z6rE': 1.0} pc3S {u'm\xe2Z6rE': 1.0} pc1P {u'm\xe2Z6rj\xf4': 1.0} pc2P {u'm\xe2Z6rje': 1.0} pc3P {u'm\xe2Z6rE': 1.0} pP {u'm\xe2Z\xe2': 1.0} ppMS {u'm\xe2Ze': 1.0} ppMP {u'm\xe2Ze': 1.0} ppFS {u'm\xe2Ze': 1.0} ppFP {u'm\xe2Ze': 1.0} ai1S {u'm\xe2ZE': 1.0} ai2S mâZe pas de patron
{} ai

In [37]:
verbes[verbes["lexeme"]=="abaisser"]

,ortho,phono,ext,cs,ms,vs,lexeme,freq,prob,case
15,abaisser,abEse,"R""",V,,inf,abaisser,1090,3.072840e-06,inf
16,abaisse,abEs,@,V,1S,pi,abaisser,1280,3.608473e-06,pi1S
17,abaisses,abEs,@,V,2S,pi,abaisser,160,4.510592e-07,pi2S
18,abaisse,abEs,@,V,3S,pi,abaisser,1280,3.608473e-06,pi3S
19,abaissons,abEsô,"z""",V,1P,pi,abaisser,20,5.638239e-08,pi1P
20,abaissez,abEse,"z""",V,2P,pi,abaisser,530,1.494133e-06,pi2P
21,abaissent,abEs,"@t""",V,3P,pi,abaisser,50,1.409560e-07,pi3P
22,abaissais,abEsE,"z""",V,1S,ii,abaisser,1,2.819120e-09,ii1S
23,abaissais,abEsE,"z""",V,2S,ii,abaisser,1,2.819120e-09,ii2S
24,abaissait,abEsE,"t""",V,3S,ii,abaisser,20,5.638239e-08,ii3S
